In [1]:
import numpy as np
import tensorflow as tf
from typing import Tuple
import os

In [4]:
class GraphModel(tf.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, graph: Tuple[np.ndarray, np.ndarray, np.ndarray]) -> tf.Tensor:
        A, E, V = graph
        
        N = V.shape[0]
        M = A.shape[1]
        
        A_full = np.zeros((N, N))
        for i in range(M):
            node_from, node_to = A[:, i]
            A_full[int(node_from), int(node_to)] = 1
        
        I = np.eye(N)
        
        D = np.diag(np.sum(A_full, axis=1))
        
        D_inv_sqrt = np.linalg.inv(np.sqrt(D + I))
        
        A_hat = D_inv_sqrt @ (A_full + I) @ D_inv_sqrt
        
        V_transformed = A_hat @ V

        V_activated = tf.nn.relu(V_transformed)

        aggregated_output = tf.reduce_sum(V_activated)
        
        return aggregated_output

    def save(self, save_path: str):
        tf.saved_model.save(self, save_path)
        print(f"Модель сохранена в {save_path}")

    @staticmethod
    def load(load_path: str) -> 'GraphModel':
        loaded_model = tf.saved_model.load(load_path)
        print(f"Модель загружена из {load_path}")
        return loaded_model

# Пример использования:

# Матрица смежности A (2 x M), где каждая колонка — это ребро: [узел_из, узел_в]
A = np.array([[0, 1, 1], [1, 2, 0]])

# Матрица эмбеддингов рёбер E (M x K)
E = np.random.randn(3, 4)  # M=3, K=4 (случайные эмбеддинги рёбер)

# Матрица эмбеддингов узлов V (N x C)
V = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])  # N=3, C=2

# Кортеж, представляющий граф
graph = (A, E, V)

# Инициализируем объект класса GraphModel
graph_model = GraphModel()

# Вызываем модель с графом (A, E, V)
output = graph_model(graph)
print(f"Результат: {output.numpy()}")

Результат: 25.266669199057848


In [6]:
save_path = 'graph_model_saved'

graph_model.save(save_path)

loaded_model = GraphModel.load(save_path)

output_loaded = loaded_model(graph)
print(f"Результат загруженной модели: {output_loaded.numpy()}")

FailedPreconditionError: graph_model_saved\variables is not a directory